# 7章　仮説と推定

## 7.1 統計的仮説検定

- 古典的な設定では、帰無仮説 H0 が基本的な立ち位置を表し対立仮説 H1 と比較さ れる
- 統計量を用いてこの帰無仮説H0を棄却できるか否かを決定する

## 7.2 事例：コイン投げ

コインをn回投げて表が出た回数Xはベルヌーイ試行にあたり、XはBinominal(n, p)の確率変数となる。これを正規分布で近似すると、、

In [43]:
from typing import Tuple
import math

def normal_approximation_to_binominal(n: int, p: float) -> Tuple[float, float]:
    # Binominal(n, p)に相当する平均μと標準偏差σを計算する
    mu = p * n
    sigma = math.sqrt(p * (1- p) * n)
    return mu, sigma

from  scratch.probability import normal_cdf

# 正規分布の累積分布関数CDF
# 変数が閾値を下回る確率はnormal cdfで表せる
normal_probability_below= normal_cdf

# 閾値を下回っていなければ、閾値より上にある
def normal_probability_above(lo: float, mu: float = 0, sigma: float = 1) -> float:
    return 1 - normal_cdf(lo, mu, sigma)
                          
# hiより小さくloより大きければ、値はその間
def normal_probability_between(lo: float, hi: float, mu: float=0, sigma: float = 1) -> float:
    return normal_cdf(hi, mu, sigma) - normal_cdf(lo, mu, sigma)

# 間になければ、範囲外
def normal_probability_outside(lo: float, hi: float, mu: float = 0, sigma: float = 1) -> float:
    return 1 - normal_probability_between(lo, hi, mu, sigma)


In [44]:
# 逆の手順（あるレベルまでの可能性に相当する区間、または平均を中心とした左右対称な領域を求める）

from scratch.probability import inverse_normal_cdf

def normal_upper_bound(probability: float, mu: float = 0, sigma: float = 1) -> float:
    # 確率P(Z <= z)となるzを返す
    return inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability: float, mu: float = 0, sigma: float = 1) -> float:
    # 確率P(Z >=z)となるzを返す
    return inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability: float, mu: float = 0, sigma: float = 1) -> Tuple[float, float]:
    # 指定された確率を包含する対象な境界を返す
    tail_probability = (1 - probability) /2 
    
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)
    return lower_bound, upper_bound
    
# コイン投げの回数をn=1000にしたときの平均と分散で正規分布の近似
mu_0, sigma_0 = normal_approximation_to_binominal(1000, 0.5)
print(mu_0, sigma_0)


500.0 15.811388300841896


In [45]:
# 実際は真にもかかわらず帰無仮説を棄却してしまう第一種の過誤をどの程度受け入れるか=有意水準　を5%を使用し
# Xが以下で与えられる区間外となりH0が棄却されるという状況について、、、

lower_bound, upper_bound = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(lower_bound, upper_bound)

469.01026640487555 530.9897335951244


## 7.3 p値

- 上記の検定をはかる別の尺度がp値
- 特定の確率でのカットオフを選ぶ代わりに、、**帰無仮説H0が真であると仮定して実際に観測された値と同等に極端な値が生じる確率を計算する**

In [46]:
def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float: 
    if x >= mu:
        return 2*normal_probability_above(x, mu, sigma)
    else:
        return 2*normal_probability_below(x, mu, sigma)
    
# 表が530出た場合は p値は0.062で5%より大きく帰無仮説は棄却されない
print( two_sided_p_value(529.5, mu_0, sigma_0))

# 表が532回出た場合。5%より小さい値となり、帰無仮説は棄却される
print(two_sided_p_value(532, mu_0, sigma_0))


0.06207721579598835
0.04298479507085862


## 7.4 信頼区間



In [47]:
# ベルヌーイ変数の平均値は平均pと次のような標準偏差の正規分布にしたがう
# math.sqrt(p * (1 - p) /1000)  

#  pの推定値
p_hat = 525 /1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)
# 0.0158

# 指定された確率を含有する境界
# pの値が次の区間に入るのは「95%の確率で信頼できる」という結論になる
normal_two_sided_bounds(0.95, mu, sigma)

(0.4940490278129096, 0.5559509721870904)

## 7.5 pハッキング

- 外れ値を適切に取り除くことでp値はおそらく0.05未満にできる
- 「p値を使った推定の枠組み」から得られる結論に対して、何らかの手を入れてしまうことがをpハッキングと呼ぶ
- データの整理は仮説を前提とせずに行う必要がある

## 7.6 事例：A/Bテストの実施

- NA人が広告Aを見てnA人がクリックした場合、広告がクリックされる確率がPAのベルヌーイ試行とみなせる
- nA/NAは平均PA、標準偏差σ=√np(1-p)の正規分布に近似できる確率変数とみなせる
- また広告Bのクリックも同様な確率となり、これらの正規分布が独立であるなら、その差も正規分布となる
- これにより標準正規分布を持つ統計量を用いてPAとPBが等しい（PA - PBが０になる）という帰無仮説を検定できる

In [54]:
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma

def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) /math.sqrt(sigma_A ** 2  + sigma_B ** 2)

#Aをクリックしたのが1000人中200人、Bをクリックしたのが1000人中180人だった場合
z = a_b_test_statistic(1000, 200, 1000, 180)
two_sided_p_value(z)
# 0.254
# この値は十分大きいため、帰無仮説を棄却できず、違いがあるという結論は導けない

#  Bをクリックした人が1000人中150人だった場合
z = a_b_test_statistic(1000, 200, 1000, 150)
print(two_sided_p_value(z))

# 両方の広告の効果が等しいとして、このクリック数の差がでる確率は0.003しか無い

0.003189699706216853


## 7.7 ベイズ推定

- 上記の手法は、検定の結果「帰無仮説が正しい場合に、これだけの違いがでる確率は0.3%しかない」
- 対して、未知のパラメータを確率変数として扱う推定の手法もある
- ベイズ推定はパラメータの事前分布に対して、観測とデータとベイズの定理を用いて更新されたパラメータの事後分布を求めることができる
- 例えば、確率が未知のパラメータの場合、事前分布として0から１の間のさまざまな値を取りうるベータ分布が頻繁に利用される